In [13]:
from ibm_watson import SpeechToTextV1, websocket
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import threading
import os

# Replace these with your IBM Watson credentials
api_key = 'your api key'
service_url = 'your api url'

# Set up the authenticator and service
authenticator = IAMAuthenticator(api_key)
speech_to_text = SpeechToTextV1(authenticator=authenticator)
speech_to_text.set_service_url(service_url)

# Custom RecognizeCallback to handle streaming results
class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):  # Corrected __init__ method
        super().__init__()
        self.transcript = []  # Initialize the transcript list

    def on_transcription(self, transcript):
        for result in transcript:
            print("Transcript:", result['transcript'])
            self.transcript.append(result['transcript'])

    def on_connected(self):
        print('Connection established!')

    def on_error(self, error):
        print(f"Error received: {error}")

    def on_inactivity_timeout(self, error):
        print(f"Inactivity timeout: {error}")

    def on_listening(self):
        print('Service is listening...')

    def on_close(self):
        print("Connection closed!")

# Function to stream audio and get transcription
def transcribe_using_websocket(file_path):
    my_callback = MyRecognizeCallback()
    
    # Open audio file in binary mode
    with open(file_path, 'rb') as audio_file:
        audio_source = AudioSource(audio_file)
        
        # Start the transcription in a new thread
        recognize_thread = threading.Thread(target=speech_to_text.recognize_using_websocket,
                                            args=(audio_source, "audio/wav", my_callback))
        recognize_thread.start()
        recognize_thread.join()  # Wait for the thread to complete

# Example usage
file_path = 'C:/Users/DELL/Jupyter notebook/audio/harvard.wav'  # Replace with your audio file path
transcribe_using_websocket(file_path)


Connection established!
Service is listening...
Transcript: the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health invest a salt pickled tastes fine with ham tacos al pastor are my favorite a zest for food is the hot cross bun 
Connection closed!
Connection closed!
